#### Note: this file is to download only the first image from a given record's URL. Works on MacOS for parallel downloading.

In [10]:
# Pytohn 3.10.5
import pandas as pd
import requests
import concurrent.futures
import json
import re
# importing os module
import os
import multiprocess as mp

from urllib.parse import urlparse
from bs4 import BeautifulSoup


taipei = pd.read_csv('taipei_image_url.csv', usecols=[0,1,2])
taipei = taipei.astype(str)

# series = taipei[taipei.columns[0], taipei.columns[1]].dropna()
print(taipei)

              Title 品名                                             URL 網頁  \
0                  寒林圖  https://digitalarchive.npm.gov.tw/Painting/Con...   
1                 群峰霽雪  https://digitalarchive.npm.gov.tw/Painting/Con...   
2                 寒江釣艇  https://digitalarchive.npm.gov.tw/Painting/Con...   
3                臨流獨坐圖  https://digitalarchive.npm.gov.tw/Painting/Con...   
4                山鷓棘雀圖  https://digitalarchive.npm.gov.tw/Painting/Con...   
..                 ...                                                ...   
280  宋高宗書孝經馬和之繪圖 (感應章）  https://digitalarchive.npm.gov.tw/Painting/Con...   
281  宋高宗書孝經馬和之繪圖 (事君章）  https://digitalarchive.npm.gov.tw/Painting/Con...   
282  宋高宗書孝經馬和之繪圖 (事君章）  https://digitalarchive.npm.gov.tw/Painting/Con...   
283  宋高宗書孝經馬和之繪圖 (喪親章）  https://digitalarchive.npm.gov.tw/Painting/Con...   
284  宋高宗書孝經馬和之繪圖 (喪親章）  https://digitalarchive.npm.gov.tw/Painting/Con...   

    objectID  
0       1198  
1       1266  
2       1223  
3         45  


#### BeautifulSoup

In [11]:
baseUrl = "https://digitalarchive.npm.gov.tw"
image_code_list = []
image_id_list = []
image_url_list = []

def run():
    for item in taipei['objectID']:
        cid = item
        url = "https://digitalarchive.npm.gov.tw/Painting/Content?pid={}&Dept=P".format(cid)
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        gallery_div = soup.find_all('div', id='gallery')
        img_list = []
        for img in gallery_div:
            img_list.append(img.find_all('img'))
    # # print(img_list[0][0])
        image_code = img_list[0][0].get('data-image-code')
        image_code_list.append(image_code)
        image_id = img_list[0][0].get('data-image-id')
        image_id_list.append(image_id)
        image_url = baseUrl + "/Image/Download" + "?imageId=" + str(image_id) + "&dept=P&cid=" + str(cid) + "&code=" + str(image_code);
        image_url_list.append(image_url)
    
    taipei['image_code'] = image_code_list
    taipei['image_id'] = image_id_list
    taipei['image_url'] = image_url_list


In [12]:
with concurrent.futures.ThreadPoolExecutor() as executor:
        # Use map to distribute the work among the threads
        executor.submit(run)

taipei.describe()
taipei.tail(20)

taipei.to_csv('taipei_image_code_id_url.csv', index=False)

In [15]:
# taipei = pd.read_csv('taipei_image_code.csv', dtype=str)
print(len(taipei))
baseUrl = "https://digitalarchive.npm.gov.tw"

# store each image to a folder
def download_file(args):
    imageURL, fileName = args
    response = requests.get(imageURL)
    if response.status_code == 200:
        path = "/Users/qum/Documents/GitHub/chinese-art/Images/taipei/" + fileName + "/"
        if not os.path.exists(path):
            os.makedirs(path)
            with open(path + fileName + ".jpg", 'wb') as f:
                f.write(response.content)

# store all images in one folder
# def download_file(args):
#     imageURL, fileName = args
#     if not os.path.exists(fileName + ".jpg"):
#         response = requests.get(imageURL)
#         if response.status_code == 200:
#             with open(fileName + ".jpg", 'wb') as f:
#                 f.write(response.content)
#     else:
#         print(fileName + ".jpg" + " already exists.")

285


#### Ready to download all images

In [16]:
downloadUrlList = []
fileNameList = []

for index, row in taipei.iterrows():
    image_id = row['image_id']
    cid = row['objectId']
    image_code = row['image_code']
    imageURL = baseUrl + "/Image/Download" + "?imageId=" + str(image_id) + "&dept=P&cid=" + str(cid) + "&code=" + str(image_code);
    downloadUrlList.append(imageURL)
    fileNameList.append(cid)

print(len(downloadUrlList))
args_list = list(zip(downloadUrlList, fileNameList))


285


#### Use Parallel Thread to download images

In [17]:


with concurrent.futures.ThreadPoolExecutor() as executor:
        # Use map to distribute the work among the threads
        executor.map(download_file, args_list)

1198.jpg already exists.
1266.jpg already exists.
1223.jpg already exists.
13943.jpg already exists.
46.jpg already exists.
13944.jpg already exists.
1300.jpg already exists.
1277.jpg already exists.
1227.jpg already exists.
13945.jpg already exists.
13946.jpg already exists.
47.jpg already exists.
1280.jpg already exists.
13947.jpg already exists.
1177.jpg already exists.
1263.jpg already exists.
1292.jpg already exists.
1311.jpg already exists.
1185.jpg already exists.
1328.jpg already exists.
39.jpg already exists.
917.jpg already exists.
918.jpg already exists.
1402.jpg already exists.
1222.jpg already exists.
13955.jpg already exists.
13954.jpg already exists.
1233.jpg already exists.
13966.jpg already exists.
13961.jpg already exists.
13962.jpg already exists.
1357.jpg already exists.
1356.jpg already exists.
14347.jpg already exists.
72.jpg already exists.
939.jpg already exists.
14346.jpg already exists.
13970.jpg already exists.
14349.jpg already exists.
13960.jpg already exis